In [ ]:
## TODO HERE!!!

In [32]:
import json
from collections import defaultdict
import torch
from vocab import Vocab
import os
import progressbar
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from collections import Counter
from shutil import copyfile

## Setup

In [5]:
object_state_types = ['isToggled', 'isBroken', 'isFilledWithLiquid', 'isDirty',
                      'isUsedUp', 'isCooked', 'ObjectTemperature', 'isSliced',
                      'isOpen', 'isPickedUp', 'mass', 'parentReceptacles']
receptacle_key = 'receptacleObjectIds'

## Args

In [33]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.data_root = '/root/media/legg/data-850-evo/'
args.data = '/root/media/legg/data-850-evo/json_data_augmentation_20200820/'
args.splits = '/root/home/legg/data_alfred/splits/sample_failed_20200820.json'

In [34]:
object_vocab = torch.load(os.path.join(args.data, 'objects_20200522.object_vocab'))
object_vocab

{'object_type': Vocab(111)}

## Hack: Simply copy the extracted_feature_states.json from full explainer pp folder to one directory up

In [35]:
with open(args.splits, 'r') as f:
    success_outpaths = json.load(f)['augmentation']
    
len(success_outpaths)
    
print('success paths = ', len(success_outpaths))

success paths =  7649


In [37]:
for task in progressbar.progressbar(success_outpaths):
    
    pp_folder = 'pp_model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2'
    src_p = os.path.join(args.data, task['task'], pp_folder, f'extracted_feature_states.json')
    dst_p = os.path.join(args.data, task['task'], f'extracted_feature_states.json')
    
    copyfile(src_p, dst_p)

100% (7649 of 7649) |####################| Elapsed Time: 0:00:11 Time:  0:00:11


## (Ignore Below: Original Code) AUGMENTATION SPLIT

In [22]:
with open(args.splits, 'r') as f:
    success_outpaths = json.load(f)['augmentation']
    
len(success_outpaths)
    
print('success paths = ', len(success_outpaths))

success paths =  7649


In [23]:
success_outpaths[0]

{'task': 'look_at_obj_in_light-BaseballBat-None-DeskLamp-301/trial_T20200814_164125_595727',
 'repeat_idx': 0,
 'full_traj_success': False,
 'collected_subgoals': 2}

In [27]:
extract_feat_dum = []
extract_feat_outpaths = []
corrected = 0

for task in progressbar.progressbar(success_outpaths):
    
    # load metadata
    metadata_outpath = os.path.join(args.data, task['task'], 'metadata_states.json')
    with open(metadata_outpath, 'r') as f:
        saved_states = json.load(f)
    
    # load traj data
    pp_folder = 'pp_model:seq2seq_per_subgoal,name:v2_epoch_40_obj_instance_enc_max_pool_dec_aux_loss_weighted_bce_1to2'
    traj_data_p = os.path.join(args.data, task['task'], pp_folder, f'demo_{task["repeat_idx"]}.json')
    with open(traj_data_p, 'r') as f:
        traj_data = json.load(f)

    # extract obj state features
    root = os.path.join(args.data, task['task'])
    shifted_states = shift_states(saved_states)
    extract_feat = extract_states_for_model(shifted_states, saved_states, root)
    extract_feat['metadata_path'] = metadata_outpath
    extract_feat['root'] = root
    
    # check if extracted features align with traj data, apply correction
    corrected += int(check_and_correct_alignments(extract_feat, traj_data))
    
    # save out
    extract_feat_outpath = metadata_outpath.replace('/metadata_states.json', '/extracted_feature_states.json')
    with open(extract_feat_outpath, 'w') as f:
        json.dump(extract_feat ,f)
    
    # copy of all output paths
    extract_feat_outpaths.append(extract_feat_outpath)
    extract_feat_dum.append(extract_feat)
    
print(len(extract_feat_dum))

N/A% (0 of 7649) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

> <ipython-input-12-768027a895ed>(30)check_and_correct_alignments()
     28             import pdb; pdb.set_trace()
     29 
---> 30         apply_alignment_correction(extracted_features, subgoal_count_specific_features)
     31 
     32         subgoal_len_features2 = [(i,len(subgoal)) for i, subgoal in enumerate(extracted_features['instance_visibile'])]

ipdb> q


BdbQuit: 

In [17]:
extract_feat_outpaths[:10]

['/root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_simple-CellPhone-None-Sofa-224/trial_T20190912_062415_255725/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_heat_then_place_in_recep-Tomato-None-DiningTable-19/trial_T20190911_051611_152502/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_cool_then_place_in_recep-Tomato-None-Microwave-20/trial_T20190911_220048_789694/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_simple-CellPhone-None-Dresser-314/trial_T20190910_153622_489437/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_and_place_simple-CreditCard-None-SideTable-313/trial_T20190913_140646_900196/extracted_feature_states.json',
 '/root/data_alfred/unlabeled_12k_20201206/seen/pick_cool_then_place_in_recep-AppleSliced-None-SinkBasin-7/trial_T20190910_135634_526097/extracted_feature_states.json',
 '/root/data_alfred/un

In [18]:
date = '20201228'
split_name = 'augmentation'
with open('/root/data_alfred/splits/extract_feat_states_{}_{}_notebook_outpaths.json'.format(date, split_name), 'w') as f:
    json.dump(extract_feat_outpaths, f)